In [10]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client.local

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


In [11]:
def find_books_by_genre(genre):
    results = db.books.find(
        {"genre": genre},
        {"_id": 0, "title": 1, "author": 1}
    )
    return list(results)


In [13]:
def average_rating_by_director():
    pipeline = [
        {"$group": {
            "_id": "$director",
            "average_rating": {"$avg": "$rating"}
        }},
        {"$sort": {"average_rating": -1}}
    ]
    results = db.movies.aggregate(pipeline)
    return list(results)


In [14]:
def recent_user_actions(user_id, limit=5):
    results = db.user_actions.find(
        {"user_id": user_id}
    ).sort("timestamp", -1).limit(limit)
    return list(results)


In [15]:
def count_books_by_year():
    pipeline = [
        {"$group": {
            "_id": "$year",
            "count": {"$sum": 1}
        }},
        {"$sort": {"count": -1}}
    ]
    results = db.books.aggregate(pipeline)
    return list(results)


In [16]:
def update_user_view_to_seen(user_id, before_date):
    result = db.user_actions.update_many(
        {
            "user_id": user_id,
            "action": "view",
            "timestamp": {"$lt": before_date}
        },
        {"$set": {"action": "seen"}}
    )
    return result.modified_count


In [17]:
if __name__ == "__main__":
    print("Fantasy 장르 책:")
    print(find_books_by_genre("fantasy"))

    print("\n감독별 평균 평점:")
    print(average_rating_by_director())

    print("\n최근 사용자 행동:")
    print(recent_user_actions(1))

    print("\n연도별 책 수:")
    print(count_books_by_year())

    print("\n행동 업데이트 수:")
    modified = update_user_view_to_seen(1, datetime(2023, 4, 13))
    print(f"{modified}개 문서가 수정되었습니다.")


Fantasy 장르 책:
[]

감독별 평균 평점:
[{'_id': 'Christopher Nolan', 'average_rating': 8.799999999999999}]

최근 사용자 행동:
[{'_id': ObjectId('67e386748dffdaf4802dbc3c'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}, {'_id': ObjectId('67e386748dffdaf4802dbc3b'), 'user_id': 1, 'action': 'click', 'timestamp': datetime.datetime(2023, 4, 12, 8, 0)}]

연도별 책 수:
[{'_id': 2002, 'count': 1}, {'_id': 2009, 'count': 1}, {'_id': 1987, 'count': 1}]

행동 업데이트 수:
1개 문서가 수정되었습니다.
